In [1]:
import matplotlib.pyplot as plt
import pickle
import os
import snap
from itertools import combinations
import numpy as np
from google.cloud import bigquery
import collections
from IPython.lib import backgroundjobs as bg
from tqdm import tqdm_notebook
import heapq
from gensim.models import Word2Vec
import node2vec
import networkx as nx

In [2]:
with open('./data/project_authors_2016_q1_k25core.pickle', 'rb') as picklefile:
    project_authors_q1 = pickle.load(picklefile)

In [16]:
dimensions=32
window=10
iterations=10
p=1
q=1
num_walks=100
walk_length = 100

In [4]:
nx_weighted = nx.Graph()
nx_weighted.add_nodes_from(project_authors_q1['author_id'].unique())
nx_weighted.add_nodes_from(project_authors_q1['project_id'].unique())
for _, row in project_authors_q1.iterrows():
    nx_weighted.add_edge(-row['project_id'], row['author_id'], weight=row['commit_times'])

In [5]:
len(nx_weighted)

7376

In [6]:
G = node2vec.Graph(nx_weighted, False, p, q)
%time G.preprocess_transition_probs()

CPU times: user 1min 49s, sys: 841 ms, total: 1min 50s
Wall time: 1min 50s


In [17]:
%time walks = G.simulate_walks(num_walks, walk_length)
walks = [map(str, walk) for walk in walks]
%time model = Word2Vec(walks, size=dimensions, window=window, min_count=0, sg=1, iter=iterations)

Walk iteration:
1 / 100
2 / 100
3 / 100
4 / 100
5 / 100
6 / 100
7 / 100
8 / 100
9 / 100
10 / 100
11 / 100
12 / 100
13 / 100
14 / 100
15 / 100
16 / 100
17 / 100
18 / 100
19 / 100
20 / 100
21 / 100
22 / 100
23 / 100
24 / 100
25 / 100
26 / 100
27 / 100
28 / 100
29 / 100
30 / 100
31 / 100
32 / 100
33 / 100
34 / 100
35 / 100
36 / 100
37 / 100
38 / 100
39 / 100
40 / 100
41 / 100
42 / 100
43 / 100
44 / 100
45 / 100
46 / 100
47 / 100
48 / 100
49 / 100
50 / 100
51 / 100
52 / 100
53 / 100
54 / 100
55 / 100
56 / 100
57 / 100
58 / 100
59 / 100
60 / 100
61 / 100
62 / 100
63 / 100
64 / 100
65 / 100
66 / 100
67 / 100
68 / 100
69 / 100
70 / 100
71 / 100
72 / 100
73 / 100
74 / 100
75 / 100
76 / 100
77 / 100
78 / 100
79 / 100
80 / 100
81 / 100
82 / 100
83 / 100
84 / 100
85 / 100
86 / 100
87 / 100
88 / 100
89 / 100
90 / 100
91 / 100
92 / 100
93 / 100
94 / 100
95 / 100
96 / 100
97 / 100
98 / 100
99 / 100
100 / 100
CPU times: user 24min 28s, sys: 3.4 s, total: 24min 32s
Wall time: 24min 35s
CPU times: user

In [8]:
len(walks)

737600

In [9]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q1 = pickle.load(picklefile)
with open('./data/2016_q2_cci-weights_kcore.pickle', 'rb') as picklefile:
    weights_q2 = pickle.load(picklefile)
edges_q1 = set(edge for edge, weight in weights_q1)
edges_q2 = set(edge for edge, weight in weights_q2)
new_edges=edges_q2.difference(edges_q1)
k = len(new_edges)

In [67]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

2548

6000 walks

In [15]:
# authors = project_authors_q1['author_id'].unique()
# pairs = list(combinations(authors, 2))
# dist = []
# for u, v in tqdm_notebook(pairs, mininterval=1):
#     if u > v:
#         u, v = v, u
#     if (u, v) not in edges_q1:
#         dist.append((model.wv.distance(str(u), str(v)), (u, v)))
# dist = sorted(dist)
topk = set(edge for weight, edge in dist[:10*k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(12657, 0.49984203459442383)

In [18]:
authors = project_authors_q1['author_id'].unique()
pairs = list(combinations(authors, 2))
dist = []
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    if (u, v) not in edges_q1:
        dist.append((model.wv.distance(str(u), str(v)), (u, v)))
dist = sorted(dist)
topk = set(edge for weight, edge in dist[:k])
len(topk.intersection(new_edges)), len(topk.intersection(new_edges))/float(k)

(3122, 0.12329199905220757)

In [11]:
len(edges_q1), k

(529154, 25322)

In [19]:
pairs = list(combinations(authors, 2))
alldist = {}
for u, v in tqdm_notebook(pairs, mininterval=1):
    if u > v:
        u, v = v, u
    alldist[(u, v)] = model.wv.distance(str(u), str(v))

In [20]:
with open('./data/node2vec-weighted-p1-q1-100-100.pickle', 'wb') as picklefile:
    pickle.dump(alldist, picklefile)